In [ ]:
# stdlib
import os

# syft absolute
import syft as sy
from syft.util.test_helpers.email_helpers import load_users
from syft.util.test_helpers.job_helpers import load_jobs
from syft.util.test_helpers.job_helpers import save_jobs

In [ ]:
ADMIN_EMAIL, ADMIN_PW = "admin2@bigquery.org", "bqpw2"
ROOT_EMAIL, ROOT_PW = "admin@bigquery.org", "bqpw"
environment = os.environ.get("ORCHESTRA_DEPLOYMENT_TYPE", "python")
low_port = os.environ.get("CLUSTER_HTTP_PORT_LOW", "9083")
print(environment, low_port)

# Launch server and login

In [ ]:
server_low = sy.orchestra.launch(
    name="bigquery-low",
    server_side_type="low",
    dev_mode=True,
    n_consumers=1,
    create_producer=True,
    port=low_port,
)

# Download Results

In [ ]:
low_client = sy.login(
    url=f"http://localhost:{low_port}", email=ADMIN_EMAIL, password=ADMIN_PW
)

In [ ]:
users = load_users(low_client)
jobs = load_jobs(users, low_client)

In [ ]:
reviewed_jobs = [job for job in jobs if job.admin_reviewed]
reviewed_jobs_should_succeed = [j for j in reviewed_jobs if j.should_succeed]
reviewed_jobs_should_fail = [j for j in reviewed_jobs if not j.should_succeed]

print(
    f"{len(reviewed_jobs)=}, {len(reviewed_jobs_should_succeed)=}, {len(reviewed_jobs_should_fail)=}"
)

In [ ]:
for job in reviewed_jobs_should_succeed:
    print(f"> Checking job: {job.job_type} {job.func_name} for user {job.user_email}")
    api_method = job.code_method
    j = api_method(blocking=False)
    res = j.wait()

    if isinstance(res, sy.SyftError):
        raise sy.SyftException(public_message="Expected success, got error")

    result = res.get()
    job.result_as_expected = True

In [ ]:
for job in reviewed_jobs_should_fail:
    print(f"> Checking job: {job.job_type} {job.func_name} for user {job.user_email}")
    api_method = job.code_method

    with sy.raises(sy.SyftException):
        j = api_method(blocking=False)
        res = j.wait()
    job.result_as_expected = True

In [ ]:
save_jobs(jobs)

In [ ]:
expected_jobs = [job for job in jobs if job.result_as_expected]
print(f"got expected_jobs: {len(expected_jobs)} == reviewed_jobs: {len(reviewed_jobs)}")
assert len(reviewed_jobs) == len(expected_jobs)

# Cleanup

In [ ]:
if environment != "remote":
    server_low.land()